In [2]:
import pandas as pd, numpy as np
import sys,os,shutil,gc,re,json,glob,math,time,random,warnings
from tqdm import tqdm
from sklearn.model_selection import StratifiedGroupKFold
import sklearn.metrics as skm
import torch
import cv2
import yaml
import albumentations as A

N_SPLITS = 5
RANDOM_STATE = 41
FOLD=0

def fix_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False

fix_seed(RANDOM_STATE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
DIR_DATA = 'data'
df = pd.read_csv(f'{DIR_DATA}/Train.csv')
CLASSES = sorted(df['class'].unique())
CLASSES = {c:i for i,c in enumerate(CLASSES)}
print(CLASSES)
df['path'] = f'{DIR_DATA}/images/'+df.Image_ID
df['category_id'] = df['class'].map(CLASSES).fillna(-1)
df['crop'] = df['class'].str.split('_').str[0]
d = df.groupby('Image_ID')['crop'].nunique()
df = pd.merge(df,pd.DataFrame({'Image_ID':d.index,'n_crop':d.values}),on='Image_ID',how='left')
df

{'Corn_Cercospora_Leaf_Spot': 0, 'Corn_Common_Rust': 1, 'Corn_Healthy': 2, 'Corn_Northern_Leaf_Blight': 3, 'Corn_Streak': 4, 'Pepper_Bacterial_Spot': 5, 'Pepper_Cercospora': 6, 'Pepper_Early_Blight': 7, 'Pepper_Fusarium': 8, 'Pepper_Healthy': 9, 'Pepper_Late_Blight': 10, 'Pepper_Leaf_Blight': 11, 'Pepper_Leaf_Curl': 12, 'Pepper_Leaf_Mosaic': 13, 'Pepper_Septoria': 14, 'Tomato_Bacterial_Spot': 15, 'Tomato_Early_Blight': 16, 'Tomato_Fusarium': 17, 'Tomato_Healthy': 18, 'Tomato_Late_Blight': 19, 'Tomato_Leaf_Curl': 20, 'Tomato_Mosaic': 21, 'Tomato_Septoria': 22}


,Image_ID,confidence,class,ymin,xmin,ymax,xmax,path,category_id,crop,n_crop
0,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,194.649671,328.803454,208.107730,341.967928,data/images/id_11543h.jpg,5,Pepper,1
1,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,149.632401,256.768914,162.910362,266.195724,data/images/id_11543h.jpg,5,Pepper,1
2,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,234.046875,327.138158,252.712993,338.876645,data/images/id_11543h.jpg,5,Pepper,1
3,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,221.277138,340.411184,238.593750,354.651316,data/images/id_11543h.jpg,5,Pepper,1
4,id_11ee1c.jpg,1.0,Pepper_Fusarium,2000.563598,989.588908,2184.252196,1401.748952,data/images/id_11ee1c.jpg,8,Pepper,1
...,...,...,...,...,...,...,...,...,...,...,...
41024,id_38x7ln.jpg,1.0,Corn_Streak,608.472222,817.741770,736.352881,997.047325,data/images/id_38x7ln.jpg,4,Corn,1
41025,id_7uqmqp.jpg,1.0,Corn_Healthy,1483.095425,818.249483,1786.752393,1152.860096,data/images/id_7uqmqp.jpg,2,Corn,1
41026,id_bnqorw.jpg,1.0,Corn_Cercospora_Leaf_Spot,943.260816,1015.079256,1007.928503,1133.177119,data/images/id_bnqorw.jpg,0,Corn,1
41027,id_m2kklk.jpg,1.0,Tomato_Septoria,1855.337090,1748.733977,2046.688525,1868.580553,data/images/id_m2kklk.jpg,22,Tomato,1


In [13]:
df.describe()

,confidence,ymin,xmin,ymax,xmax,category_id,n_crop
count,41029.0,41029.000000,41029.000000,41029.000000,41029.000000,41029.000000,41029.000000
mean,1.0,1096.986018,1665.664444,1321.627860,1927.684518,11.325501,1.145604
std,0.0,682.484829,973.855104,714.825614,1026.212994,8.086232,0.352713
min,1.0,0.000000,0.000000,30.998991,17.225673,0.000000,1.000000
25%,1.0,566.046875,886.775362,744.048622,1066.148385,4.000000,1.000000
50%,1.0,994.930609,1568.155180,1246.348948,1838.952447,13.000000,1.000000
75%,1.0,1551.876065,2363.446657,1810.713525,2663.181818,19.000000,1.000000
max,1.0,4295.942029,5908.600181,4480.000000,6567.427798,22.000000,2.000000


In [14]:
def split_data(df):

    gkf  = StratifiedGroupKFold(n_splits=N_SPLITS,shuffle=True,random_state=RANDOM_STATE)

    df['fold'] = -1

    for fold_id, (train_index, test_index) in enumerate(gkf.split(df,y=df['class'],groups=df.Image_ID)):

        df.loc[test_index,'fold'] = fold_id

    return df


df = split_data(df)
df

,Image_ID,confidence,class,ymin,xmin,ymax,xmax,path,category_id,crop,n_crop,fold
0,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,194.649671,328.803454,208.107730,341.967928,data/images/id_11543h.jpg,5,Pepper,1,3
1,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,149.632401,256.768914,162.910362,266.195724,data/images/id_11543h.jpg,5,Pepper,1,3
2,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,234.046875,327.138158,252.712993,338.876645,data/images/id_11543h.jpg,5,Pepper,1,3
3,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,221.277138,340.411184,238.593750,354.651316,data/images/id_11543h.jpg,5,Pepper,1,3
4,id_11ee1c.jpg,1.0,Pepper_Fusarium,2000.563598,989.588908,2184.252196,1401.748952,data/images/id_11ee1c.jpg,8,Pepper,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
41024,id_38x7ln.jpg,1.0,Corn_Streak,608.472222,817.741770,736.352881,997.047325,data/images/id_38x7ln.jpg,4,Corn,1,2
41025,id_7uqmqp.jpg,1.0,Corn_Healthy,1483.095425,818.249483,1786.752393,1152.860096,data/images/id_7uqmqp.jpg,2,Corn,1,4
41026,id_bnqorw.jpg,1.0,Corn_Cercospora_Leaf_Spot,943.260816,1015.079256,1007.928503,1133.177119,data/images/id_bnqorw.jpg,0,Corn,1,2
41027,id_m2kklk.jpg,1.0,Tomato_Septoria,1855.337090,1748.733977,2046.688525,1868.580553,data/images/id_m2kklk.jpg,22,Tomato,1,2


In [15]:
df['class'].value_counts()

,count
class,
Corn_Cercospora_Leaf_Spot,6538
Tomato_Septoria,6483
Tomato_Late_Blight,3918
Corn_Streak,3180
Tomato_Healthy,2872
Pepper_Septoria,2295
Pepper_Leaf_Mosaic,2113
Tomato_Early_Blight,1946
Pepper_Bacterial_Spot,1943


In [16]:
cls = ['Pepper_Early_Blight', 'Pepper_Late_Blight', 'Pepper_Septoria']
df_pepper = df[df['class'].isin(cls)].copy()
df = df[~((df.n_crop>1)&(df.crop=='Tomato'))].copy()

CLASSES = sorted(df['class'].unique())
CLASSES = {c:i for i,c in enumerate(CLASSES)}
print(len(CLASSES),CLASSES)



23 {'Corn_Cercospora_Leaf_Spot': 0, 'Corn_Common_Rust': 1, 'Corn_Healthy': 2, 'Corn_Northern_Leaf_Blight': 3, 'Corn_Streak': 4, 'Pepper_Bacterial_Spot': 5, 'Pepper_Cercospora': 6, 'Pepper_Early_Blight': 7, 'Pepper_Fusarium': 8, 'Pepper_Healthy': 9, 'Pepper_Late_Blight': 10, 'Pepper_Leaf_Blight': 11, 'Pepper_Leaf_Curl': 12, 'Pepper_Leaf_Mosaic': 13, 'Pepper_Septoria': 14, 'Tomato_Bacterial_Spot': 15, 'Tomato_Early_Blight': 16, 'Tomato_Fusarium': 17, 'Tomato_Healthy': 18, 'Tomato_Late_Blight': 19, 'Tomato_Leaf_Curl': 20, 'Tomato_Mosaic': 21, 'Tomato_Septoria': 22}


In [17]:
df['class'].value_counts()

,count
class,
Corn_Cercospora_Leaf_Spot,6538
Tomato_Septoria,4166
Tomato_Late_Blight,3343
Corn_Streak,3180
Tomato_Healthy,2872
Pepper_Septoria,2295
Pepper_Leaf_Mosaic,2113
Pepper_Bacterial_Spot,1943
Tomato_Early_Blight,1845


In [18]:
for fold in range(N_SPLITS):
  d=df[df.fold==fold]['class'].value_counts().to_dict()
  print(fold,d)

0 {'Corn_Cercospora_Leaf_Spot': 1326, 'Tomato_Septoria': 804, 'Tomato_Late_Blight': 773, 'Corn_Streak': 636, 'Tomato_Healthy': 563, 'Pepper_Septoria': 551, 'Pepper_Bacterial_Spot': 445, 'Tomato_Early_Blight': 366, 'Pepper_Leaf_Mosaic': 327, 'Corn_Common_Rust': 315, 'Corn_Healthy': 302, 'Pepper_Leaf_Curl': 272, 'Tomato_Fusarium': 176, 'Pepper_Healthy': 130, 'Pepper_Late_Blight': 117, 'Pepper_Leaf_Blight': 112, 'Tomato_Bacterial_Spot': 97, 'Pepper_Cercospora': 91, 'Pepper_Fusarium': 84, 'Corn_Northern_Leaf_Blight': 57, 'Tomato_Leaf_Curl': 54, 'Tomato_Mosaic': 24, 'Pepper_Early_Blight': 15}
1 {'Corn_Cercospora_Leaf_Spot': 1369, 'Tomato_Septoria': 759, 'Tomato_Late_Blight': 743, 'Corn_Streak': 690, 'Pepper_Leaf_Mosaic': 489, 'Tomato_Healthy': 480, 'Pepper_Bacterial_Spot': 395, 'Corn_Common_Rust': 369, 'Pepper_Leaf_Curl': 324, 'Pepper_Septoria': 309, 'Corn_Healthy': 285, 'Tomato_Early_Blight': 273, 'Tomato_Fusarium': 199, 'Pepper_Healthy': 150, 'Tomato_Leaf_Curl': 112, 'Pepper_Fusarium': 10

In [39]:
# scores = pd.read_csv('scores_1024_wtd_1280_cv.csv')
# df1 = pd.merge(scores,df,on='Image_ID').sort_values(by='score')
# df_candidates = df1[df1.score>0.8].copy()
# df_candidates.Image_ID.nunique(),len(df_candidates)

In [45]:
# candidate_image_ids = {}
# for Image_ID, d in df_candidates.groupby('Image_ID'):
#   candidate_image_ids[Image_ID] = d.score.mean()

# Offline Augmentation

In [44]:
# @title
candidate_image_ids = {'id_11ee1c.jpg': 1.0, 'id_11zffz.jpg': 1.0, 'id_127uhh.jpg': 1.0, 'id_12q33h.jpg': 0.9166666666666666, 'id_13cfty.jpg': 0.8333333333333333, 'id_15q5ff.jpg': 1.0, 'id_19het6.jpg': 1.0, 'id_19obez.jpg': 1.0, 'id_1arv86.jpg': 1.0, 'id_1bh5f5.jpg': 0.8110239433768845, 'id_1bjbim.jpg': 1.0, 'id_1csdxd.jpg': 0.8983516483516485, 'id_1es1zh.jpg': 1.0, 'id_1fdmyb.jpg': 0.8242424242424242, 'id_1g6n1s.jpg': 0.8333333333333334, 'id_1gkerv.jpg': 1.0, 'id_1h9rga.jpg': 1.0, 'id_1hf6im.jpg': 0.9640625000000002, 'id_1hx47n.jpg': 0.8955582232893156, 'id_1kmy6r.jpg': 1.0, 'id_1lp2yq.jpg': 1.0, 'id_1ml83k.jpg': 0.8333333333333333, 'id_1nd3hv.jpg': 1.0, 'id_1ouore.jpg': 1.0, 'id_1pq8fc.jpg': 1.0, 'id_1r3rzf.jpg': 0.8793604056761951, 'id_1rq1ti.jpg': 1.0, 'id_1u2r1y.jpg': 1.0, 'id_1v5awt.jpg': 0.875, 'id_1x29p5.jpg': 1.0, 'id_1xbhq1.jpg': 0.8333333333333333, 'id_1z1whi.jpg': 0.8095238095238096, 'id_1zmth9.jpg': 1.0, 'id_21qqu3.jpg': 1.0, 'id_23h185.jpg': 0.95, 'id_23kin4.jpg': 0.8167995746943114, 'id_245r78.jpg': 1.0, 'id_247m8o.jpg': 0.888888888888889, 'id_24c7u4.jpg': 0.8125, 'id_24ggcs.jpg': 1.0, 'id_26hk3g.jpg': 1.0, 'id_26qct9.jpg': 1.0, 'id_27cebp.jpg': 0.8112703905091448, 'id_27m8xg.jpg': 1.0, 'id_2a5ohr.jpg': 0.9272727272727274, 'id_2c2cvd.jpg': 1.0, 'id_2ccmfn.jpg': 0.8333333333333334, 'id_2d9mch.jpg': 1.0, 'id_2dm56o.jpg': 0.8666666666666667, 'id_2dmimn.jpg': 1.0, 'id_2dtbw9.jpg': 1.0, 'id_2eztxw.jpg': 0.8333333333333333, 'id_2fvida.jpg': 1.0, 'id_2hrihc.jpg': 1.0, 'id_2j8x6h.jpg': 0.9428571428571428, 'id_2plxex.jpg': 1.0, 'id_2qbotq.jpg': 1.0, 'id_2qduig.jpg': 0.876923076923077, 'id_2ri37e.jpg': 1.0, 'id_2rjath.jpg': 1.0, 'id_2rxuoj.jpg': 1.0, 'id_2t6f1k.jpg': 1.0, 'id_2uhcjo.jpg': 0.8825222396650968, 'id_2upgy2.jpg': 0.8333333333333333, 'id_2uq2jz.jpg': 0.8039750557413599, 'id_2vv55m.jpg': 0.8095238095238096, 'id_2w1vds.jpg': 1.0, 'id_2z8vob.jpg': 1.0, 'id_32e7um.jpg': 0.9454545454545451, 'id_33ubpc.jpg': 0.8140226090978487, 'id_34phbc.jpg': 1.0, 'id_35652s.jpg': 1.0, 'id_36g8pe.jpg': 1.0, 'id_3bem21.jpg': 1.0, 'id_3c174b.jpg': 0.9761904761904762, 'id_3c3pjf.jpg': 1.0, 'id_3c9wz6.jpg': 1.0, 'id_3edz79.jpg': 0.875, 'id_3evymc.jpg': 0.861111111111111, 'id_3fjjp9.jpg': 1.0, 'id_3g7ezo.jpg': 1.0, 'id_3gyhwe.jpg': 1.0, 'id_3h9xal.jpg': 1.0, 'id_3i93h6.jpg': 1.0, 'id_3kak6l.jpg': 1.0, 'id_3lnrol.jpg': 0.8336134453781513, 'id_3lp2pz.jpg': 0.9714285714285714, 'id_3n1jzi.jpg': 1.0, 'id_3nqzn2.jpg': 0.8500000000000001, 'id_3oftql.jpg': 1.0, 'id_3p35u4.jpg': 0.8609768907563027, 'id_3q1eqa.jpg': 1.0, 'id_3r7znt.jpg': 0.9461538461538463, 'id_3uz99o.jpg': 1.0, 'id_3xa5zj.jpg': 1.0, 'id_3ymyzt.jpg': 1.0, 'id_3yp46v.jpg': 1.0, 'id_43v1wa.jpg': 1.0, 'id_45ubar.jpg': 0.9245054320501892, 'id_47zr2e.jpg': 1.0, 'id_493hsj.jpg': 1.0, 'id_4acfej.jpg': 0.9400666673393944, 'id_4affyg.jpg': 0.8719047619047618, 'id_4b85zu.jpg': 0.8932832278481013, 'id_4c5jyk.jpg': 1.0, 'id_4cfgnj.jpg': 0.8106285538450869, 'id_4glxan.jpg': 0.9835164835164836, 'id_4guxuw.jpg': 1.0, 'id_4hu5hz.jpg': 0.8617915302134318, 'id_4ik598.jpg': 1.0, 'id_4j54bj.jpg': 0.8937173937173937, 'id_4knbj9.jpg': 1.0, 'id_4lqeqt.jpg': 0.888888888888889, 'id_4m4yk2.jpg': 1.0, 'id_4mjlua.jpg': 1.0, 'id_4n1359.jpg': 0.9074159663865548, 'id_4ndy4w.jpg': 1.0, 'id_4oedpx.jpg': 1.0, 'id_4p3e91.jpg': 1.0, 'id_4qocyz.jpg': 0.8500000000000001, 'id_4s5a5h.jpg': 1.0, 'id_4slhnz.jpg': 0.9444444444444445, 'id_4tiulj.jpg': 0.9725274725274722, 'id_4tp4qc.jpg': 1.0, 'id_4u5vko.jpg': 1.0, 'id_4ubugj.jpg': 1.0, 'id_4ukm2v.jpg': 1.0, 'id_4vryyz.jpg': 1.0, 'id_4y2hmd.jpg': 0.8433333333333334, 'id_4yhwe7.jpg': 1.0, 'id_4z9z65.jpg': 1.0, 'id_5185b6.jpg': 0.8269536019536018, 'id_51z6xb.jpg': 0.8611111111111112, 'id_5235yv.jpg': 0.8284090909090908, 'id_541pe2.jpg': 0.8208333333333334, 'id_54m32a.jpg': 1.0, 'id_54x869.jpg': 1.0, 'id_55jdeb.jpg': 1.0, 'id_598as9.jpg': 0.9166666666666666, 'id_59ryp4.jpg': 1.0, 'id_59vkzv.jpg': 1.0, 'id_5ai47a.jpg': 1.0, 'id_5akdjy.jpg': 0.9488235294117648, 'id_5b8sdf.jpg': 1.0, 'id_5cttnc.jpg': 0.8999999999999999, 'id_5d7gwf.jpg': 0.9761904761904762, 'id_5extvz.jpg': 0.975, 'id_5ftemg.jpg': 1.0, 'id_5fvdhf.jpg': 0.8802129547471161, 'id_5fyp85.jpg': 0.95, 'id_5idnae.jpg': 1.0, 'id_5ifbme.jpg': 1.0, 'id_5kwaqk.jpg': 1.0, 'id_5mtk38.jpg': 0.8419467787114847, 'id_5mxacz.jpg': 0.8381410256410257, 'id_5nuo78.jpg': 0.8999999999999999, 'id_5pw54p.jpg': 1.0, 'id_5r5zgs.jpg': 1.0, 'id_5rube5.jpg': 1.0, 'id_5sjc7p.jpg': 0.8222222222222222, 'id_5tozve.jpg': 1.0, 'id_5uxpm7.jpg': 0.9583333333333331, 'id_5y78xu.jpg': 0.9444444444444445, 'id_5y8dom.jpg': 0.8152872778079356, 'id_5zrl45.jpg': 0.869724025974026, 'id_5zxene.jpg': 1.0, 'id_61dn3e.jpg': 0.9166666666666666, 'id_61gig2.jpg': 1.0, 'id_63mfpf.jpg': 1.0, 'id_646sp8.jpg': 0.916527922651047, 'id_663pgs.jpg': 0.8907407407407406, 'id_665kfk.jpg': 0.8228571428571427, 'id_673zig.jpg': 0.8186371048006268, 'id_67lcxs.jpg': 0.906140512631385, 'id_67skc7.jpg': 0.8333333333333333, 'id_67wlqq.jpg': 0.8222222222222221, 'id_69rqhd.jpg': 1.0, 'id_6aa259.jpg': 1.0, 'id_6agfg7.jpg': 1.0, 'id_6csghm.jpg': 1.0, 'id_6cuw7a.jpg': 1.0, 'id_6d43zx.jpg': 1.0, 'id_6dkdsn.jpg': 0.8333333333333333, 'id_6fqzl6.jpg': 0.9553571428571427, 'id_6hmbg3.jpg': 0.8333333333333334, 'id_6huazk.jpg': 0.8666666666666667, 'id_6ifv3f.jpg': 0.8690476190476191, 'id_6kwsue.jpg': 1.0, 'id_6l1oiu.jpg': 1.0, 'id_6ljb99.jpg': 0.9571428571428572, 'id_6nmpic.jpg': 1.0, 'id_6rt8j5.jpg': 1.0, 'id_6rtvu3.jpg': 0.9682539682539684, 'id_6s5qxp.jpg': 1.0, 'id_6sd21d.jpg': 0.8394966173603943, 'id_6tbuvd.jpg': 0.8338076275576277, 'id_6ul155.jpg': 1.0, 'id_6va3yw.jpg': 0.8051948051948052, 'id_6vplm5.jpg': 1.0, 'id_6w3r22.jpg': 1.0, 'id_6wauce.jpg': 0.8381936245572611, 'id_6winav.jpg': 0.8126666666666666, 'id_6yius2.jpg': 0.8333333333333333, 'id_6yr6lo.jpg': 1.0, 'id_6zk2dv.jpg': 1.0, 'id_71mqrk.jpg': 0.9572649572649573, 'id_71rtrt.jpg': 1.0, 'id_72cqqu.jpg': 0.8881456707543663, 'id_72xpeg.jpg': 1.0, 'id_739ljd.jpg': 0.8753885003885002, 'id_73rp97.jpg': 0.829004329004329, 'id_75vjvr.jpg': 1.0, 'id_76l367.jpg': 0.8142857142857142, 'id_772l8v.jpg': 1.0, 'id_77opu3.jpg': 1.0, 'id_788qwa.jpg': 1.0, 'id_78p6dy.jpg': 0.9333333333333332, 'id_79jim4.jpg': 1.0, 'id_7bktuv.jpg': 1.0, 'id_7bnydv.jpg': 1.0, 'id_7d73j5.jpg': 0.8188235294117646, 'id_7d9c5g.jpg': 1.0, 'id_7dzted.jpg': 1.0, 'id_7fdsl8.jpg': 1.0, 'id_7fs4aj.jpg': 1.0, 'id_7hjbdr.jpg': 1.0, 'id_7hpl1o.jpg': 1.0, 'id_7jzy9y.jpg': 0.9166666666666666, 'id_7ko81k.jpg': 1.0, 'id_7kvxkx.jpg': 1.0, 'id_7nejo8.jpg': 0.814423076923077, 'id_7nnu7t.jpg': 0.8993055555555556, 'id_7oxlec.jpg': 0.87874767744333, 'id_7q8zpo.jpg': 1.0, 'id_7ra4m1.jpg': 0.95, 'id_7rsxuf.jpg': 0.8950007356984104, 'id_7uq986.jpg': 1.0, 'id_7uqmqp.jpg': 0.9249847374847374, 'id_7w8ff8.jpg': 1.0, 'id_7x98iy.jpg': 1.0, 'id_7xneap.jpg': 1.0, 'id_81c4ow.jpg': 0.8545338258784105, 'id_82tsb1.jpg': 0.8155118334759053, 'id_83oxrr.jpg': 0.8040935672514619, 'id_841y8l.jpg': 0.9727272727272729, 'id_842sys.jpg': 0.8526785714285714, 'id_84aa7l.jpg': 1.0, 'id_84fm7g.jpg': 0.85, 'id_84mbmu.jpg': 1.0, 'id_84vl9c.jpg': 0.8538359788359788, 'id_88c99i.jpg': 0.9305555555555555, 'id_88fi4h.jpg': 0.8485499462943071, 'id_89k2xp.jpg': 1.0, 'id_89ka78.jpg': 0.8617424242424242, 'id_89yuz9.jpg': 1.0, 'id_8bhvmj.jpg': 1.0, 'id_8cpl6h.jpg': 0.85, 'id_8dct94.jpg': 1.0, 'id_8dompf.jpg': 1.0, 'id_8hcbxi.jpg': 1.0, 'id_8i57ur.jpg': 0.8472222222222223, 'id_8nglc3.jpg': 0.9538461538461538, 'id_8otbl6.jpg': 0.8768037518037521, 'id_8pt1h8.jpg': 0.9682539682539684, 'id_8qrtcx.jpg': 0.9464285714285714, 'id_8qx5td.jpg': 1.0, 'id_8t9jjy.jpg': 1.0, 'id_8tkdrv.jpg': 1.0, 'id_8ud2ni.jpg': 1.0, 'id_8uea7t.jpg': 0.8333333333333333, 'id_8v427h.jpg': 1.0, 'id_8vaknc.jpg': 1.0, 'id_8x6x5n.jpg': 1.0, 'id_8ymq74.jpg': 1.0, 'id_8z4kz1.jpg': 0.8333333333333334, 'id_91i1pt.jpg': 0.9304152637485971, 'id_91qf2z.jpg': 0.9426684426684431, 'id_93rq6u.jpg': 0.8611111111111112, 'id_95hqyb.jpg': 1.0, 'id_96sqfc.jpg': 1.0, 'id_98a2rn.jpg': 0.9555555555555555, 'id_98bccl.jpg': 1.0, 'id_999cuc.jpg': 1.0, 'id_99tpad.jpg': 0.8333333333333333, 'id_9aiwx9.jpg': 0.8311111111111111, 'id_9akb2t.jpg': 0.8666666666666668, 'id_9d9a7t.jpg': 0.9333333333333336, 'id_9g55wb.jpg': 0.8609847370657253, 'id_9jkyok.jpg': 0.8372209317331268, 'id_9k42as.jpg': 0.9285714285714287, 'id_9ne8iv.jpg': 1.0, 'id_9omhs4.jpg': 0.885, 'id_9ovbei.jpg': 1.0, 'id_9rg5eh.jpg': 0.8666666666666667, 'id_9rqfpa.jpg': 1.0, 'id_9shnv1.jpg': 1.0, 'id_9u8ukt.jpg': 0.8333333333333333, 'id_9voogp.jpg': 1.0, 'id_9wh21s.jpg': 1.0, 'id_9x9rwf.jpg': 0.8720238095238094, 'id_9xkh7s.jpg': 1.0, 'id_9xz995.jpg': 0.9769230769230768, 'id_9ztlgb.jpg': 0.8333333333333333, 'id_a12gcz.jpg': 0.8181628329451756, 'id_a2ha4v.jpg': 1.0, 'id_a3f87o.jpg': 0.9, 'id_a4ku63.jpg': 0.8333333333333334, 'id_a4mjtf.jpg': 1.0, 'id_a5l87e.jpg': 0.898326210826211, 'id_a5lym6.jpg': 1.0, 'id_a652k1.jpg': 1.0, 'id_a6k8yd.jpg': 0.8537027665317138, 'id_a7devn.jpg': 0.9166666666666666, 'id_a7f6ih.jpg': 0.8333333333333334, 'id_a825vj.jpg': 1.0, 'id_a8f43c.jpg': 1.0, 'id_a9shyg.jpg': 1.0, 'id_aacz3x.jpg': 0.8333333333333333, 'id_abb3a1.jpg': 0.8226010101010102, 'id_adkhru.jpg': 1.0, 'id_aeb7it.jpg': 0.9924242424242423, 'id_ai98wa.jpg': 0.875, 'id_ajzraz.jpg': 0.9367346938775507, 'id_ak1l92.jpg': 0.8100649350649347, 'id_akkii9.jpg': 0.885, 'id_akqtau.jpg': 1.0, 'id_al886s.jpg': 1.0, 'id_alqc7t.jpg': 1.0, 'id_amyul7.jpg': 0.9, 'id_anek6l.jpg': 1.0, 'id_aoaqpy.jpg': 1.0, 'id_apwnyk.jpg': 1.0, 'id_aqxdl8.jpg': 1.0, 'id_armkkb.jpg': 1.0, 'id_assxt9.jpg': 0.8964646464646463, 'id_at127j.jpg': 1.0, 'id_avgg52.jpg': 1.0, 'id_avlmnv.jpg': 1.0, 'id_awdwyi.jpg': 1.0, 'id_axfwao.jpg': 1.0, 'id_ay85js.jpg': 1.0, 'id_azbboc.jpg': 0.8005050505050505, 'id_b11ovu.jpg': 0.8680555555555556, 'id_b2jdbn.jpg': 0.861111111111111, 'id_b2jl8x.jpg': 1.0, 'id_b44tum.jpg': 1.0, 'id_b4ckrl.jpg': 0.8450249398599471, 'id_b4uqln.jpg': 0.8500000000000001, 'id_b61lxk.jpg': 1.0, 'id_b6qhax.jpg': 0.8238304093567251, 'id_b73f3c.jpg': 0.8041666666666668, 'id_b88kza.jpg': 1.0, 'id_b8fur5.jpg': 1.0, 'id_b8if16.jpg': 1.0, 'id_bae3bn.jpg': 0.9166666666666666, 'id_banous.jpg': 0.8473634390301059, 'id_bazjub.jpg': 0.8422619047619048, 'id_bbcd7b.jpg': 0.934409025318116, 'id_bbtodu.jpg': 0.8999999999999999, 'id_bct8vx.jpg': 0.8610061813186811, 'id_bdffkz.jpg': 1.0, 'id_bdt6rs.jpg': 0.9028571428571428, 'id_bfhswy.jpg': 1.0, 'id_bfysgm.jpg': 1.0, 'id_bg8xlh.jpg': 0.9476301476301477, 'id_bhbfbp.jpg': 1.0, 'id_biswso.jpg': 0.9047619047619048, 'id_bkoqa8.jpg': 0.9367346938775507, 'id_bkskuy.jpg': 0.8111446246061631, 'id_blv53n.jpg': 0.8507326007326006, 'id_bnqorw.jpg': 0.8743567774817774, 'id_bnx8y2.jpg': 0.8666666666666667, 'id_bqkr1n.jpg': 1.0, 'id_brbab5.jpg': 0.8928571428571428, 'id_bs7i79.jpg': 0.8214285714285713, 'id_bves57.jpg': 0.9376623376623379, 'id_bw958v.jpg': 0.9722222222222224, 'id_bwfn5e.jpg': 1.0, 'id_bxbmod.jpg': 0.9444444444444446, 'id_byni8h.jpg': 1.0, 'id_c17pxb.jpg': 1.0, 'id_c3gt8n.jpg': 1.0, 'id_c66lkl.jpg': 0.8333333333333333, 'id_c6g65r.jpg': 0.8214285714285715, 'id_c73p4l.jpg': 0.8869832275004688, 'id_c7lhzl.jpg': 1.0, 'id_c8kzdi.jpg': 0.9583333333333334, 'id_c9k7xm.jpg': 1.0, 'id_c9xvh6.jpg': 1.0, 'id_cb8y9a.jpg': 1.0, 'id_cc5qn7.jpg': 1.0, 'id_cd9osv.jpg': 1.0, 'id_cfazwd.jpg': 1.0, 'id_cffivw.jpg': 1.0, 'id_ch54ki.jpg': 0.8666666666666668, 'id_chb581.jpg': 0.8521095938375352, 'id_chreep.jpg': 0.9152241081818546, 'id_ckctgg.jpg': 0.9189332025716467, 'id_ckoime.jpg': 1.0, 'id_cmh4id.jpg': 0.9166666666666666, 'id_cnu55w.jpg': 1.0, 'id_crkoiv.jpg': 1.0, 'id_crzr4c.jpg': 0.8711111111111111, 'id_cs9sxo.jpg': 0.8333333333333333, 'id_csbzeh.jpg': 0.9473715622612682, 'id_csr3jb.jpg': 0.8095238095238096, 'id_cui9oi.jpg': 1.0, 'id_cv11kz.jpg': 0.8484848484848485, 'id_cvtbxs.jpg': 0.8333333333333333, 'id_cyet9c.jpg': 1.0, 'id_cygxx4.jpg': 0.9166666666666666, 'id_d2ro5x.jpg': 1.0, 'id_d3ngz2.jpg': 0.85, 'id_d5ijd7.jpg': 1.0, 'id_d5u1c7.jpg': 1.0, 'id_d74usz.jpg': 0.9333333333333332, 'id_d8seit.jpg': 0.9166666666666666, 'id_dbvbfx.jpg': 1.0, 'id_dcj1ys.jpg': 0.8666666666666667, 'id_dcr7ym.jpg': 1.0, 'id_dd277p.jpg': 0.8738970588235294, 'id_ddicmu.jpg': 0.9166666666666666, 'id_df8chr.jpg': 1.0, 'id_dfk7b7.jpg': 1.0, 'id_dfson3.jpg': 1.0, 'id_dg692l.jpg': 1.0, 'id_dh7euo.jpg': 1.0, 'id_dibinr.jpg': 1.0, 'id_djt62z.jpg': 1.0, 'id_djufka.jpg': 0.9, 'id_dk4l2i.jpg': 1.0, 'id_dkhf6d.jpg': 0.8716931216931217, 'id_dl1hqg.jpg': 1.0, 'id_dlwx73.jpg': 1.0, 'id_dnqtm3.jpg': 1.0, 'id_doj57h.jpg': 0.8163265306122448, 'id_dphhg5.jpg': 1.0, 'id_dpy1bf.jpg': 1.0, 'id_dsdvui.jpg': 1.0, 'id_dtjesd.jpg': 1.0, 'id_dtoknw.jpg': 1.0, 'id_dva7tg.jpg': 0.8454545454545455, 'id_dwiivj.jpg': 0.8333333333333334, 'id_dwmsq5.jpg': 0.8928571428571428, 'id_dxexa5.jpg': 1.0, 'id_e76paa.jpg': 0.8032094889237749, 'id_e7duth.jpg': 0.9166666666666666, 'id_e82at1.jpg': 1.0, 'id_e8t9at.jpg': 1.0, 'id_e953bc.jpg': 1.0, 'id_e9p8ir.jpg': 0.8123294398018505, 'id_eapzsw.jpg': 1.0, 'id_ebglje.jpg': 1.0, 'id_ecfju2.jpg': 0.8190018087559069, 'id_efii1o.jpg': 1.0, 'id_efvee5.jpg': 0.8946605689103917, 'id_eh7iq5.jpg': 1.0, 'id_eh9ry9.jpg': 0.8266115431027713, 'id_ehyzk9.jpg': 0.861111111111111, 'id_eiehmh.jpg': 1.0, 'id_ekbr4a.jpg': 1.0, 'id_ekvu47.jpg': 0.9666666666666666, 'id_eny197.jpg': 1.0, 'id_eor7qh.jpg': 1.0, 'id_eppiag.jpg': 1.0, 'id_epx4lf.jpg': 0.8318523581681475, 'id_erh6tn.jpg': 1.0, 'id_errbp4.jpg': 1.0, 'id_esqa8h.jpg': 1.0, 'id_ettfp7.jpg': 1.0, 'id_eu8p1r.jpg': 1.0, 'id_euiy4c.jpg': 0.8888294711824123, 'id_ev7v6b.jpg': 1.0, 'id_evb1l1.jpg': 1.0, 'id_evizba.jpg': 0.8333333333333333, 'id_evr7ia.jpg': 1.0, 'id_exijwk.jpg': 1.0, 'id_exl8r9.jpg': 1.0, 'id_exm47q.jpg': 1.0, 'id_ey6q1u.jpg': 1.0, 'id_eyjtj7.jpg': 0.9428571428571428, 'id_ezki3j.jpg': 1.0, 'id_ezpesd.jpg': 0.9444444444444444, 'id_f2eecs.jpg': 1.0, 'id_f2og8i.jpg': 0.914423076923077, 'id_f3jwqi.jpg': 0.8333333333333334, 'id_f4dn4b.jpg': 1.0, 'id_f4vjex.jpg': 1.0, 'id_f5qqdt.jpg': 1.0, 'id_f8nvd6.jpg': 0.9457720588235294, 'id_f92bjc.jpg': 1.0, 'id_f9euw6.jpg': 0.8022727272727275, 'id_f9pdfq.jpg': 1.0, 'id_f9w8zp.jpg': 1.0, 'id_f9x1d7.jpg': 0.8755208333333332, 'id_faekqm.jpg': 1.0, 'id_fb1vkw.jpg': 1.0, 'id_fd2iyi.jpg': 0.8333333333333333, 'id_fd32yu.jpg': 0.9166666666666666, 'id_fd9zr3.jpg': 1.0, 'id_fda5vn.jpg': 1.0, 'id_fdq7kk.jpg': 1.0, 'id_fdrisg.jpg': 1.0, 'id_ffd3ky.jpg': 1.0, 'id_fgdau5.jpg': 0.8280917534477903, 'id_fgnt4q.jpg': 1.0, 'id_fk48dv.jpg': 0.8527777777777779, 'id_fk6qvn.jpg': 1.0, 'id_fkm33l.jpg': 0.8333333333333333, 'id_fko42w.jpg': 0.8095238095238096, 'id_fngwt4.jpg': 1.0, 'id_fnwe1t.jpg': 1.0, 'id_fotm5o.jpg': 1.0, 'id_fqdv2k.jpg': 0.8511217948717947, 'id_frjrhd.jpg': 0.8303571428571428, 'id_frwaj7.jpg': 1.0, 'id_fshttj.jpg': 1.0, 'id_ft8a3d.jpg': 1.0, 'id_ftsqt4.jpg': 0.85, 'id_ftyfpq.jpg': 1.0, 'id_fuvrb9.jpg': 1.0, 'id_fvndll.jpg': 0.8833333333333332, 'id_fxi6aw.jpg': 0.8039772727272726, 'id_fxlm1s.jpg': 1.0, 'id_fzaa9v.jpg': 0.9166666666666666, 'id_fzitlu.jpg': 0.9861111111111112, 'id_g1bkif.jpg': 0.875, 'id_g1fvam.jpg': 0.9205960161842516, 'id_g1wl1n.jpg': 0.807070707070707, 'id_g287mt.jpg': 1.0, 'id_g2ugi1.jpg': 0.8611111111111112, 'id_g3gz7w.jpg': 1.0, 'id_g4l84s.jpg': 0.9285714285714287, 'id_g4s485.jpg': 0.9057304686263964, 'id_g5ibaf.jpg': 0.925, 'id_g5mvid.jpg': 1.0, 'id_g6xcho.jpg': 0.9480519480519478, 'id_g6zazk.jpg': 0.91, 'id_g7iwht.jpg': 0.8369230769230768, 'id_g9mgsh.jpg': 1.0, 'id_g9qffw.jpg': 0.9146478521478526, 'id_gbux4z.jpg': 1.0, 'id_gbyu96.jpg': 0.8012477718360071, 'id_gddusf.jpg': 0.8333333333333333, 'id_gfwey7.jpg': 1.0, 'id_ghf4kc.jpg': 0.9666666666666668, 'id_ghw7mx.jpg': 0.9464285714285713, 'id_gjg3nc.jpg': 0.8016666666666665, 'id_gjh3c4.jpg': 0.9226062399139322, 'id_glvjty.jpg': 1.0, 'id_gotaw4.jpg': 0.8224752394789815, 'id_gpvt6l.jpg': 0.9366666666666668, 'id_gqkg9c.jpg': 0.8333333333333334, 'id_grgfo8.jpg': 1.0, 'id_gshoms.jpg': 1.0, 'id_gtpchm.jpg': 1.0, 'id_gv4bvo.jpg': 1.0, 'id_gv5uvb.jpg': 1.0, 'id_gv8k2p.jpg': 1.0, 'id_gvwtab.jpg': 0.8214285714285713, 'id_gwbhww.jpg': 1.0, 'id_gx7ete.jpg': 1.0, 'id_gxawoj.jpg': 0.8333333333333333, 'id_gygm3m.jpg': 1.0, 'id_gypo1e.jpg': 1.0, 'id_h1gj24.jpg': 1.0, 'id_h49xbh.jpg': 0.8333333333333333, 'id_h5xb5n.jpg': 0.9583333333333336, 'id_h67z4p.jpg': 1.0, 'id_h6birf.jpg': 0.8666666666666667, 'id_h6wkra.jpg': 0.9038712483688562, 'id_h8jrox.jpg': 0.85, 'id_hb4tx1.jpg': 1.0, 'id_hcrn2p.jpg': 0.9835164835164836, 'id_heknrz.jpg': 1.0, 'id_hewayi.jpg': 0.9444444444444446, 'id_hh1xrv.jpg': 0.867984693877551, 'id_hhotvf.jpg': 0.8333333333333334, 'id_hi46sm.jpg': 0.942193675889328, 'id_hkwpac.jpg': 0.95, 'id_hl77fo.jpg': 0.888888888888889, 'id_hn3pen.jpg': 1.0, 'id_hn8o3e.jpg': 1.0, 'id_ho73di.jpg': 1.0, 'id_hoolfn.jpg': 1.0, 'id_hpacs7.jpg': 0.8032653061224492, 'id_hq5b47.jpg': 1.0, 'id_hrg76m.jpg': 1.0, 'id_hrrbns.jpg': 1.0, 'id_htet9a.jpg': 1.0, 'id_htnrvb.jpg': 0.8333333333333333, 'id_huz2gf.jpg': 1.0, 'id_hw5gih.jpg': 1.0, 'id_hwhdx9.jpg': 0.8518518518518519, 'id_hzgtgh.jpg': 1.0, 'id_hzjk5z.jpg': 1.0, 'id_hzpeb9.jpg': 0.85, 'id_hzr4dw.jpg': 0.9166666666666666, 'id_i1jpbi.jpg': 0.8869248035914701, 'id_i22z3b.jpg': 1.0, 'id_i258fi.jpg': 0.8323082248315894, 'id_i28qhj.jpg': 1.0, 'id_i3k8s7.jpg': 1.0, 'id_i3pxxe.jpg': 0.8833333333333332, 'id_i3r282.jpg': 0.885, 'id_i43z5j.jpg': 1.0, 'id_i6np9d.jpg': 0.8333333333333333, 'id_i836oq.jpg': 1.0, 'id_i89blp.jpg': 1.0, 'id_i9utfc.jpg': 1.0, 'id_i9zfcq.jpg': 1.0, 'id_iapvud.jpg': 0.8125457875457874, 'id_iczzud.jpg': 0.8666666666666667, 'id_idt543.jpg': 1.0, 'id_iem8fs.jpg': 0.8970551378446117, 'id_ietjhz.jpg': 1.0, 'id_ihkh8c.jpg': 0.83245196567565, 'id_iiaum4.jpg': 1.0, 'id_ik355l.jpg': 1.0, 'id_ikiq9h.jpg': 0.854971738152516, 'id_ikpwxh.jpg': 0.9861111111111112, 'id_iky132.jpg': 1.0, 'id_im3q1u.jpg': 1.0, 'id_in54vb.jpg': 0.9166666666666666, 'id_ini6ub.jpg': 1.0, 'id_inoyxd.jpg': 0.8666666666666668, 'id_iqjtfc.jpg': 0.8747852021954106, 'id_iqnoc1.jpg': 0.8333333333333333, 'id_iqsxnp.jpg': 0.8100961538461537, 'id_irtmmu.jpg': 0.8116176632229385, 'id_ittqrl.jpg': 1.0, 'id_iu48bm.jpg': 1.0, 'id_ivfy5q.jpg': 0.8833616138763198, 'id_ivwe5c.jpg': 0.9481907894736842, 'id_iwefif.jpg': 0.9298499872870585, 'id_ixay3r.jpg': 0.9416666666666669, 'id_izmss2.jpg': 1.0, 'id_izo92g.jpg': 1.0, 'id_izu7eu.jpg': 1.0, 'id_j11k4i.jpg': 0.8450000000000001, 'id_j241yx.jpg': 1.0, 'id_j2xxyj.jpg': 1.0, 'id_j2y3xz.jpg': 1.0, 'id_j7454q.jpg': 0.8433333333333334, 'id_j77xns.jpg': 0.9136577708006282, 'id_j7qgue.jpg': 1.0, 'id_j8g5tt.jpg': 1.0, 'id_j8wo7s.jpg': 1.0, 'id_jacbpn.jpg': 1.0, 'id_jbkqqz.jpg': 1.0, 'id_jc27iu.jpg': 1.0, 'id_jcabvg.jpg': 1.0, 'id_jctki3.jpg': 1.0, 'id_jeskdq.jpg': 1.0, 'id_jffs9g.jpg': 0.8372153554427162, 'id_jfk1l3.jpg': 0.820673076923077, 'id_jflfyg.jpg': 1.0, 'id_jfvqjt.jpg': 0.8625687642671341, 'id_jiwy7s.jpg': 1.0, 'id_jiy8pg.jpg': 0.8333333333333334, 'id_jk4j4f.jpg': 0.8155988455988455, 'id_jlmiga.jpg': 0.8440087145969499, 'id_jmsnwy.jpg': 1.0, 'id_jmuvb5.jpg': 1.0, 'id_jmv3s3.jpg': 0.8625, 'id_jmz1zp.jpg': 1.0, 'id_jmznat.jpg': 0.904349816849817, 'id_jq82kj.jpg': 0.8333333333333333, 'id_jqc6ng.jpg': 0.8089318723679608, 'id_jrsoyi.jpg': 0.9253582674635307, 'id_jsbabh.jpg': 1.0, 'id_jtnbco.jpg': 0.9909090909090909, 'id_jvxyyj.jpg': 0.8754629629629631, 'id_jwb7lg.jpg': 1.0, 'id_jweiy8.jpg': 1.0, 'id_jwqyo1.jpg': 1.0, 'id_jxc4sx.jpg': 1.0, 'id_k1ahnz.jpg': 0.9166666666666666, 'id_k5mnqn.jpg': 1.0, 'id_k79193.jpg': 1.0, 'id_k791ju.jpg': 0.85, 'id_k7rj9h.jpg': 1.0, 'id_k8e253.jpg': 1.0, 'id_k8napj.jpg': 1.0, 'id_k9cdrt.jpg': 1.0, 'id_k9q6ro.jpg': 1.0, 'id_ka3bc2.jpg': 1.0, 'id_ka3wh5.jpg': 1.0, 'id_kak3u9.jpg': 0.880644528342738, 'id_kbckjo.jpg': 0.8523809523809524, 'id_kdon6y.jpg': 0.8952380952380953, 'id_kdw294.jpg': 1.0, 'id_khj4pf.jpg': 1.0, 'id_khl6ok.jpg': 0.8893726348660907, 'id_khsl7m.jpg': 0.9668067226890753, 'id_kih5zm.jpg': 1.0, 'id_kihbkc.jpg': 1.0, 'id_kijg5e.jpg': 0.8999999999999999, 'id_kjtzko.jpg': 0.8928030303030303, 'id_klg5yn.jpg': 0.8145363408521302, 'id_klzkmg.jpg': 1.0, 'id_knzsrc.jpg': 0.9166666666666666, 'id_koak73.jpg': 0.9655674210701383, 'id_kov1vd.jpg': 1.0, 'id_kp4ubd.jpg': 1.0, 'id_kpm1rw.jpg': 0.9023809523809522, 'id_kpsmh3.jpg': 0.9508295625942688, 'id_kpzkpn.jpg': 1.0, 'id_ksw3r6.jpg': 1.0, 'id_kt6r59.jpg': 1.0, 'id_ktgxtw.jpg': 0.8360119047619049, 'id_kxjl5m.jpg': 0.9188296592875378, 'id_kxxjsz.jpg': 1.0, 'id_l1fgl9.jpg': 1.0, 'id_l21zfx.jpg': 0.8482783882783881, 'id_l3ecnk.jpg': 1.0, 'id_l5aqsm.jpg': 1.0, 'id_l7do8e.jpg': 1.0, 'id_lalkke.jpg': 0.8999999999999999, 'id_lb7fss.jpg': 0.8211681963349026, 'id_lbgf7j.jpg': 1.0, 'id_lc7i97.jpg': 0.95, 'id_lcf9ni.jpg': 0.8667484236834724, 'id_lch318.jpg': 0.95, 'id_leb2oz.jpg': 0.8995388669301714, 'id_lhaow3.jpg': 1.0, 'id_ljuunj.jpg': 1.0, 'id_lk2mrd.jpg': 1.0, 'id_lkf6td.jpg': 1.0, 'id_llup1k.jpg': 1.0, 'id_lnflro.jpg': 1.0, 'id_lnhs7y.jpg': 0.8666666666666667, 'id_lnr27g.jpg': 1.0, 'id_lnrtsb.jpg': 0.8790174471992654, 'id_lpfgef.jpg': 1.0, 'id_lqiw7k.jpg': 1.0, 'id_lqt3ke.jpg': 1.0, 'id_lraa92.jpg': 1.0, 'id_lrjgw3.jpg': 0.9396081349206348, 'id_lruuze.jpg': 0.9040404040404042, 'id_lsjnew.jpg': 1.0, 'id_lsmyod.jpg': 1.0, 'id_lsw2r8.jpg': 1.0, 'id_lt4u9h.jpg': 0.8541666666666666, 'id_lv1c9z.jpg': 1.0, 'id_lyplwn.jpg': 1.0, 'id_m1m4fm.jpg': 0.8399505256648113, 'id_m2698y.jpg': 0.9028571428571428, 'id_m2yb6q.jpg': 1.0, 'id_m36z4y.jpg': 1.0, 'id_m395f5.jpg': 0.8095238095238096, 'id_m4ngqh.jpg': 0.9111111111111112, 'id_m5vxlq.jpg': 1.0, 'id_m6i6gn.jpg': 0.8214646464646465, 'id_m6simq.jpg': 1.0, 'id_m6zx11.jpg': 0.8333333333333334, 'id_m79tuv.jpg': 0.85, 'id_m8xfyu.jpg': 0.9761904761904762, 'id_m9fmwy.jpg': 1.0, 'id_m9kq37.jpg': 1.0, 'id_mbecwj.jpg': 1.0, 'id_mbtq4p.jpg': 0.9403243815008524, 'id_meie6x.jpg': 1.0, 'id_mf9vik.jpg': 1.0, 'id_mhwkrw.jpg': 1.0, 'id_mi2zpk.jpg': 1.0, 'id_migelu.jpg': 1.0, 'id_miu6op.jpg': 0.8333333333333333, 'id_mjjsko.jpg': 1.0, 'id_mkka1v.jpg': 1.0, 'id_mkso1m.jpg': 1.0, 'id_mlmvc3.jpg': 0.9697658862876256, 'id_mlq6zh.jpg': 1.0, 'id_mlqnf2.jpg': 0.818328664799253, 'id_mlr998.jpg': 1.0, 'id_mmdhoo.jpg': 1.0, 'id_mn12rm.jpg': 1.0, 'id_mnj4x1.jpg': 1.0, 'id_mnz2wu.jpg': 1.0, 'id_mobdnb.jpg': 1.0, 'id_mpi5ww.jpg': 1.0, 'id_mrm4h7.jpg': 1.0, 'id_msmsgg.jpg': 1.0, 'id_msr9ny.jpg': 1.0, 'id_mtksxl.jpg': 0.8409090909090908, 'id_mtmmvt.jpg': 1.0, 'id_mvdelh.jpg': 1.0, 'id_mx66ft.jpg': 0.9666666666666668, 'id_mxvb9l.jpg': 1.0, 'id_my2mey.jpg': 1.0, 'id_my4y63.jpg': 1.0, 'id_my95ut.jpg': 1.0, 'id_n1n1e1.jpg': 1.0, 'id_n23l6b.jpg': 0.8214285714285715, 'id_n23leb.jpg': 1.0, 'id_n2y5my.jpg': 1.0, 'id_n3mhvf.jpg': 1.0, 'id_n3swre.jpg': 1.0, 'id_n71kyz.jpg': 0.8333333333333333, 'id_n8qwl6.jpg': 0.9166666666666666, 'id_n977hi.jpg': 1.0, 'id_nc129b.jpg': 0.8833333333333333, 'id_nco7tp.jpg': 0.894929631771737, 'id_nczo3o.jpg': 1.0, 'id_ndik9e.jpg': 0.8555555555555555, 'id_nf8zu3.jpg': 1.0, 'id_nh4jj1.jpg': 1.0, 'id_nhl5ai.jpg': 0.925, 'id_ni1vxz.jpg': 0.9135531135531137, 'id_ni2uci.jpg': 0.8362193362193361, 'id_njyeu9.jpg': 1.0, 'id_nk8moa.jpg': 0.9166666666666666, 'id_nk9n22.jpg': 1.0, 'id_nm24en.jpg': 0.8333333333333333, 'id_nmv9jv.jpg': 1.0, 'id_nouzj6.jpg': 0.8125, 'id_noxvf3.jpg': 1.0, 'id_noyfg5.jpg': 0.8928571428571428, 'id_nr7k15.jpg': 1.0, 'id_nt3va5.jpg': 0.8333333333333333, 'id_nukr2u.jpg': 1.0, 'id_nwa4at.jpg': 0.8333333333333333, 'id_nx52io.jpg': 0.95, 'id_nxto2q.jpg': 1.0, 'id_nyserf.jpg': 1.0, 'id_o263an.jpg': 1.0, 'id_o28fu4.jpg': 1.0, 'id_o2mws3.jpg': 1.0, 'id_o2y2oz.jpg': 1.0, 'id_o3y96r.jpg': 1.0, 'id_o4vcld.jpg': 1.0, 'id_o5p5m4.jpg': 1.0, 'id_o92ipq.jpg': 1.0, 'id_oac7nn.jpg': 0.8333333333333334, 'id_oamz1l.jpg': 1.0, 'id_ofl6rm.jpg': 1.0, 'id_og8jis.jpg': 0.8666666666666667, 'id_ogrh85.jpg': 1.0, 'id_ohxwo5.jpg': 0.8285714285714284, 'id_oidzjk.jpg': 1.0, 'id_olrrlr.jpg': 0.8333333333333333, 'id_oq6hgl.jpg': 0.8999999999999999, 'id_orvhit.jpg': 0.9502840909090908, 'id_ovi4hi.jpg': 1.0, 'id_ox23ke.jpg': 1.0, 'id_ox81mw.jpg': 0.8666666666666667, 'id_oydvcx.jpg': 1.0, 'id_ozwev5.jpg': 0.9166666666666666, 'id_p1dwr8.jpg': 1.0, 'id_p1vw8l.jpg': 0.8333333333333334, 'id_p29lp9.jpg': 0.8888888888888891, 'id_p2hmw7.jpg': 0.8285714285714284, 'id_p61h7v.jpg': 1.0, 'id_p63oce.jpg': 0.9523809523809524, 'id_p6pfeh.jpg': 0.8333333333333333, 'id_p7awh7.jpg': 0.925, 'id_p9coy6.jpg': 0.8999999999999999, 'id_p9ik43.jpg': 0.9523809523809524, 'id_pa32uu.jpg': 0.9457070707070705, 'id_pb15pe.jpg': 1.0, 'id_pb92az.jpg': 1.0, 'id_pbae42.jpg': 1.0, 'id_pbavn7.jpg': 1.0, 'id_pbl92x.jpg': 1.0, 'id_pbo197.jpg': 0.8461657390228818, 'id_pbz86j.jpg': 0.9365079365079365, 'id_pe4jll.jpg': 0.8006396588486141, 'id_pg284v.jpg': 1.0, 'id_pg3on5.jpg': 0.8333333333333334, 'id_pgpr3j.jpg': 1.0, 'id_picwbo.jpg': 0.8888888888888888, 'id_pigm12.jpg': 0.8201044017710686, 'id_pindnw.jpg': 1.0, 'id_plhxe9.jpg': 1.0, 'id_po4vtm.jpg': 0.8101705960161847, 'id_ptepyp.jpg': 0.9583333333333336, 'id_pu434z.jpg': 1.0, 'id_pu7wai.jpg': 0.8537254285954887, 'id_puisj5.jpg': 0.842735042735043, 'id_pv8eo1.jpg': 1.0, 'id_pvboe6.jpg': 0.8319332509869594, 'id_pvcf7o.jpg': 0.8277777777777776, 'id_pvuwgw.jpg': 1.0, 'id_py6rl7.jpg': 0.9186067247146792, 'id_pzfaun.jpg': 1.0, 'id_q1x2ce.jpg': 1.0, 'id_q26uuk.jpg': 0.9046757164404224, 'id_q2ynq1.jpg': 1.0, 'id_q31pf3.jpg': 1.0, 'id_q43d6a.jpg': 1.0, 'id_q4dwhy.jpg': 1.0, 'id_q4hzip.jpg': 0.8278928278928279, 'id_q718aq.jpg': 0.8109492271646772, 'id_q7ykrv.jpg': 0.8570707070707073, 'id_q9isi5.jpg': 1.0, 'id_qb38un.jpg': 0.925, 'id_qbierc.jpg': 1.0, 'id_qcpjjy.jpg': 1.0, 'id_qcq88e.jpg': 1.0, 'id_qd2wz1.jpg': 1.0, 'id_qdjbb1.jpg': 1.0, 'id_qdkgnv.jpg': 0.8125, 'id_qfzg39.jpg': 0.9818181818181817, 'id_qg66yg.jpg': 0.8333333333333333, 'id_qidw57.jpg': 0.8933333333333331, 'id_qjny3p.jpg': 1.0, 'id_qjvrdn.jpg': 1.0, 'id_qkvptj.jpg': 1.0, 'id_ql9ge8.jpg': 0.9807692307692305, 'id_qmated.jpg': 1.0, 'id_qp8do2.jpg': 1.0, 'id_qppqnn.jpg': 0.9164941338854381, 'id_qsz2jc.jpg': 1.0, 'id_qtkdbx.jpg': 0.8928571428571428, 'id_quvng9.jpg': 0.9028571428571428, 'id_qxcewv.jpg': 1.0, 'id_qzdcuw.jpg': 1.0, 'id_qzj84z.jpg': 1.0, 'id_r1wsf9.jpg': 0.86, 'id_r2gxkq.jpg': 0.9642857142857143, 'id_r3dosz.jpg': 0.8095238095238096, 'id_r3hmjt.jpg': 0.9610389610389611, 'id_r4st7w.jpg': 1.0, 'id_r5xggp.jpg': 1.0, 'id_r76bki.jpg': 1.0, 'id_r78o22.jpg': 0.9772727272727271, 'id_r7ekoi.jpg': 1.0, 'id_r9sxmg.jpg': 1.0, 'id_r9tf3c.jpg': 1.0, 'id_rcelat.jpg': 1.0, 'id_rdbt4i.jpg': 0.8611111111111112, 'id_rddhdn.jpg': 1.0, 'id_re4hem.jpg': 0.9333333333333332, 'id_rgfxjv.jpg': 1.0, 'id_rgkqyh.jpg': 0.8433333333333334, 'id_rj2asg.jpg': 0.8500000000000001, 'id_rk7af1.jpg': 0.9166666666666666, 'id_rkfpw9.jpg': 1.0, 'id_rmyrrn.jpg': 1.0, 'id_rp6vwo.jpg': 1.0, 'id_rperu8.jpg': 0.885, 'id_rq5gjx.jpg': 1.0, 'id_rq9l4x.jpg': 1.0, 'id_rqnmd9.jpg': 1.0, 'id_rqqrkc.jpg': 0.8100585817691082, 'id_rrfn3a.jpg': 0.8541666666666666, 'id_rrwv8h.jpg': 1.0, 'id_rskvvj.jpg': 1.0, 'id_rt8kjb.jpg': 0.8999999999999999, 'id_rviwhn.jpg': 0.8541666666666666, 'id_rx3gn6.jpg': 1.0, 'id_rxt35e.jpg': 0.8402777777777779, 'id_rybt2k.jpg': 0.9102564102564102, 'id_s1mp6m.jpg': 1.0, 'id_s2baba.jpg': 0.8500000000000001, 'id_s5d55q.jpg': 0.9621212121212124, 'id_s8hn3v.jpg': 1.0, 'id_s8qrb9.jpg': 0.8666666666666668, 'id_s9x2bl.jpg': 1.0, 'id_saerrl.jpg': 1.0, 'id_saqld2.jpg': 0.9392857142857146, 'id_sbs9nr.jpg': 0.9909090909090909, 'id_sccvwo.jpg': 0.9396483542170775, 'id_scyo2t.jpg': 1.0, 'id_se9vac.jpg': 1.0, 'id_sf5geo.jpg': 0.8668831168831169, 'id_sgl748.jpg': 1.0, 'id_sgq1hc.jpg': 0.8333333333333334, 'id_sgz6ou.jpg': 0.8333333333333334, 'id_shufr8.jpg': 1.0, 'id_sipusp.jpg': 0.9166666666666666, 'id_sj2yex.jpg': 0.8333333333333333, 'id_sjlptt.jpg': 0.8500000000000001, 'id_sjyv7u.jpg': 0.9249999999999999, 'id_sk2wh1.jpg': 1.0, 'id_skn3au.jpg': 1.0, 'id_slitfi.jpg': 1.0, 'id_smsf93.jpg': 0.86, 'id_srflwr.jpg': 1.0, 'id_ss726i.jpg': 0.8541666666666666, 'id_ssak7s.jpg': 0.9333333333333332, 'id_ssrnr4.jpg': 1.0, 'id_st2mci.jpg': 1.0, 'id_stp94v.jpg': 0.8353494623655914, 'id_svs9mw.jpg': 0.875, 'id_szn87a.jpg': 1.0, 'id_szujki.jpg': 1.0, 'id_szwsuc.jpg': 0.8928571428571428, 'id_szwyhl.jpg': 0.9333333333333332, 'id_t2s72h.jpg': 1.0, 'id_t3k4fk.jpg': 1.0, 'id_t6rdr7.jpg': 1.0, 'id_t6z9fr.jpg': 0.9166666666666666, 'id_t74mpc.jpg': 1.0, 'id_tcm577.jpg': 1.0, 'id_tdt3un.jpg': 0.8244500139236982, 'id_temvio.jpg': 1.0, 'id_tfgn1m.jpg': 1.0, 'id_tft4xu.jpg': 1.0, 'id_tfud8z.jpg': 1.0, 'id_thue8a.jpg': 1.0, 'id_tifn7i.jpg': 1.0, 'id_tiqy52.jpg': 0.9445100354191264, 'id_tj5zjm.jpg': 0.8597563255457992, 'id_tkimdy.jpg': 1.0, 'id_tlm6kf.jpg': 0.8242424242424242, 'id_tlt5ch.jpg': 0.9246031746031744, 'id_tmgu1i.jpg': 0.9338235294117648, 'id_tnk51c.jpg': 0.8333333333333334, 'id_to2ru7.jpg': 1.0, 'id_tovw3z.jpg': 0.8928571428571428, 'id_tpqfxl.jpg': 0.8333333333333333, 'id_tq2zj4.jpg': 1.0, 'id_tqao8s.jpg': 1.0, 'id_trbqsi.jpg': 1.0, 'id_ttxgan.jpg': 1.0, 'id_tucyhe.jpg': 1.0, 'id_tuetub.jpg': 0.8395833333333333, 'id_tuyr57.jpg': 1.0, 'id_tvddpv.jpg': 1.0, 'id_tw9dxw.jpg': 1.0, 'id_twfmjc.jpg': 0.8437001594896332, 'id_tyoqb2.jpg': 0.8766233766233765, 'id_tzniia.jpg': 1.0, 'id_u1suml.jpg': 1.0, 'id_u1z43b.jpg': 0.8333333333333333, 'id_u2llog.jpg': 1.0, 'id_u3lf7o.jpg': 1.0, 'id_u5erpo.jpg': 0.8747684770609108, 'id_u5f4hd.jpg': 1.0, 'id_u9xzgd.jpg': 1.0, 'id_ua29ku.jpg': 1.0, 'id_ucag33.jpg': 0.9711111111111106, 'id_ucptyl.jpg': 0.8333333333333333, 'id_ud8hjk.jpg': 1.0, 'id_udk3fj.jpg': 0.989010989010989, 'id_udqjem.jpg': 0.8304093567251464, 'id_ue2xsl.jpg': 1.0, 'id_uf5zap.jpg': 1.0, 'id_ufdo7m.jpg': 1.0, 'id_ufn534.jpg': 1.0, 'id_ufo3oi.jpg': 1.0, 'id_ufw4gj.jpg': 0.8333333333333334, 'id_uhusub.jpg': 0.8492063492063492, 'id_uio8b9.jpg': 0.95, 'id_uk6swt.jpg': 0.9861111111111112, 'id_uln367.jpg': 1.0, 'id_umlghs.jpg': 1.0, 'id_umsrfj.jpg': 1.0, 'id_un9ohf.jpg': 0.9666666666666668, 'id_uoyhso.jpg': 1.0, 'id_uppdqk.jpg': 0.9166666666666669, 'id_uq1qmc.jpg': 0.880952380952381, 'id_uq67eh.jpg': 1.0, 'id_ursoid.jpg': 0.8448863636363637, 'id_ut5dio.jpg': 1.0, 'id_utyn3b.jpg': 0.8033333333333335, 'id_uu15my.jpg': 1.0, 'id_uuqlm9.jpg': 1.0, 'id_uur2n3.jpg': 1.0, 'id_uw5xyn.jpg': 1.0, 'id_uy5c3i.jpg': 1.0, 'id_uzdqxx.jpg': 0.8023418192599224, 'id_uzz4uf.jpg': 1.0, 'id_v1rusc.jpg': 1.0, 'id_v1wbt7.jpg': 0.9305555555555555, 'id_v2ckis.jpg': 0.9249999999999999, 'id_v2drp3.jpg': 0.94375, 'id_v4v8v4.jpg': 0.8790090913644559, 'id_v4xsiv.jpg': 1.0, 'id_v56h8h.jpg': 0.8907109893399149, 'id_v57yp9.jpg': 1.0, 'id_v5mvyi.jpg': 0.8333333333333333, 'id_v7dwvf.jpg': 1.0, 'id_v7siij.jpg': 1.0, 'id_va5o5j.jpg': 1.0, 'id_vd6zni.jpg': 0.8683041029766694, 'id_vdo7mf.jpg': 0.879251700680272, 'id_vendg2.jpg': 1.0, 'id_vg3o32.jpg': 0.8460284183554624, 'id_vgu7hy.jpg': 1.0, 'id_vhfyt3.jpg': 0.8885897435897439, 'id_vhno3k.jpg': 1.0, 'id_vikaig.jpg': 0.8333333333333334, 'id_vjq9dy.jpg': 1.0, 'id_vjwn6l.jpg': 0.8634057971014493, 'id_vk1rw9.jpg': 1.0, 'id_vklod6.jpg': 0.8666666666666667, 'id_vm7tyc.jpg': 0.898148148148148, 'id_vnzn55.jpg': 0.8412698412698414, 'id_vqsao7.jpg': 1.0, 'id_vqwqus.jpg': 1.0, 'id_vrnofe.jpg': 1.0, 'id_vs14n4.jpg': 1.0, 'id_vs3ebu.jpg': 1.0, 'id_vudee1.jpg': 0.8333333333333333, 'id_vwpckq.jpg': 0.9166666666666666, 'id_vx6cnm.jpg': 1.0, 'id_vxaj7n.jpg': 1.0, 'id_vzh4pw.jpg': 0.953753591954023, 'id_w14fp3.jpg': 1.0, 'id_w67h9k.jpg': 1.0, 'id_w6x3q5.jpg': 1.0, 'id_w7jdb6.jpg': 1.0, 'id_w841o2.jpg': 1.0, 'id_w8hi9a.jpg': 0.9477124183006537, 'id_w9d39u.jpg': 0.9270833333333334, 'id_wbt5aj.jpg': 1.0, 'id_wcyw3l.jpg': 1.0, 'id_wd65so.jpg': 1.0, 'id_wdbu6w.jpg': 0.9423076923076922, 'id_wett4o.jpg': 0.853599787022247, 'id_wfunso.jpg': 0.9285714285714287, 'id_wgjrpg.jpg': 1.0, 'id_wheg34.jpg': 0.9513888888888888, 'id_whiykc.jpg': 0.8269230769230769, 'id_whmpmq.jpg': 1.0, 'id_wi7l91.jpg': 0.9666666666666668, 'id_wimy3r.jpg': 0.8671903804256744, 'id_wk5td7.jpg': 1.0, 'id_wkmx5h.jpg': 1.0, 'id_wkt9fu.jpg': 1.0, 'id_wl8h57.jpg': 1.0, 'id_wnbg1m.jpg': 0.8669130197635544, 'id_wnd85s.jpg': 1.0, 'id_wp1j9l.jpg': 1.0, 'id_wqgkg2.jpg': 1.0, 'id_wr5mvb.jpg': 1.0, 'id_wrxwph.jpg': 0.8073045191546212, 'id_wt4fsp.jpg': 0.993589743589744, 'id_wtge5p.jpg': 1.0, 'id_wtshwf.jpg': 1.0, 'id_wv7snt.jpg': 0.9633333333333333, 'id_wwu6pq.jpg': 0.8333333333333334, 'id_wxjq1l.jpg': 0.8628571428571428, 'id_wyxqtm.jpg': 1.0, 'id_wzdxwg.jpg': 1.0, 'id_wzqlg1.jpg': 1.0, 'id_x23s1g.jpg': 0.9028571428571428, 'id_x29g4o.jpg': 0.8054371408318777, 'id_x3pb38.jpg': 1.0, 'id_x4ioky.jpg': 1.0, 'id_x58xhc.jpg': 0.8121618809821053, 'id_x5ti7w.jpg': 0.8578311556418389, 'id_x5xnzf.jpg': 1.0, 'id_x5yq14.jpg': 1.0, 'id_x6fyno.jpg': 1.0, 'id_x6uuhn.jpg': 0.8714285714285714, 'id_x858dm.jpg': 1.0, 'id_x8oq5y.jpg': 1.0, 'id_x9q139.jpg': 1.0, 'id_xbjckb.jpg': 0.8666666666666667, 'id_xbkb8r.jpg': 0.8490002154708036, 'id_xdo4en.jpg': 1.0, 'id_xeudya.jpg': 0.9166666666666666, 'id_xfxpxv.jpg': 0.8398934398934401, 'id_xilpll.jpg': 1.0, 'id_xiurz6.jpg': 0.9333333333333332, 'id_xjgyql.jpg': 0.95, 'id_xl26h9.jpg': 0.8625235902139652, 'id_xls7w3.jpg': 1.0, 'id_xopkc1.jpg': 0.8136037068295532, 'id_xopy32.jpg': 0.8333333333333333, 'id_xq7w2h.jpg': 1.0, 'id_xreva1.jpg': 0.8631308810949532, 'id_xrlaoj.jpg': 1.0, 'id_xs51vj.jpg': 1.0, 'id_xsmzzt.jpg': 0.8128502415458936, 'id_xt1zk7.jpg': 0.9238782051282052, 'id_xv36zt.jpg': 0.8766233766233765, 'id_xv53hj.jpg': 0.8743767949156348, 'id_xybki8.jpg': 0.8500000000000001, 'id_xytr7s.jpg': 0.8333333333333333, 'id_xzio5g.jpg': 0.85, 'id_y2x8tw.jpg': 1.0, 'id_y4yhzs.jpg': 0.8371099316044456, 'id_y5rekk.jpg': 0.9166666666666666, 'id_y5wwgz.jpg': 0.8166666666666665, 'id_y6968m.jpg': 0.95, 'id_yb54vf.jpg': 1.0, 'id_ybfdb6.jpg': 1.0, 'id_yfv7f9.jpg': 0.8975000000000001, 'id_yg1zyk.jpg': 1.0, 'id_yh4y2p.jpg': 0.9523809523809524, 'id_yhmrl1.jpg': 1.0, 'id_ykngok.jpg': 1.0, 'id_ym6pab.jpg': 0.8142504554269261, 'id_ymwp6o.jpg': 0.8757575757575757, 'id_ynnm1m.jpg': 1.0, 'id_yo9tjq.jpg': 1.0, 'id_yohbv2.jpg': 1.0, 'id_yoyjoa.jpg': 0.8720991970991973, 'id_ypfci1.jpg': 1.0, 'id_ypt19m.jpg': 1.0, 'id_yr9uva.jpg': 0.861111111111111, 'id_yrryl7.jpg': 0.8954475308641974, 'id_ysm3lu.jpg': 0.8839285714285713, 'id_ywqbhw.jpg': 1.0, 'id_yx13rc.jpg': 0.8333333333333334, 'id_yxhp49.jpg': 0.828544061302682, 'id_yyf8qx.jpg': 1.0, 'id_yyrrnc.jpg': 0.9871794871794872, 'id_z3e3y5.jpg': 1.0, 'id_z3u74r.jpg': 1.0, 'id_z4859f.jpg': 0.9245120652128511, 'id_z486u7.jpg': 0.8333333333333333, 'id_z553xf.jpg': 1.0, 'id_z5blbr.jpg': 1.0, 'id_z6ezmp.jpg': 1.0, 'id_z6ihz6.jpg': 0.9777777777777781, 'id_z7iy82.jpg': 0.8333333333333333, 'id_zd4qdk.jpg': 1.0, 'id_zdd66x.jpg': 1.0, 'id_zdztj2.jpg': 1.0, 'id_zepvqj.jpg': 0.8459207459207457, 'id_zfrgs5.jpg': 1.0, 'id_zjgtyu.jpg': 0.8968348902559429, 'id_zm7k25.jpg': 1.0, 'id_zmn2xt.jpg': 0.9173280423280424, 'id_znm3bu.jpg': 1.0, 'id_zojjxe.jpg': 0.8999999999999999, 'id_zp1zgc.jpg': 1.0, 'id_zrm6q3.jpg': 1.0, 'id_zsw5qz.jpg': 1.0, 'id_zt5wso.jpg': 0.9333333333333332, 'id_zwo7fo.jpg': 1.0, 'id_zyohbi.jpg': 0.9013333333333331, 'id_zysbnj.jpg': 0.9478458049886618, 'id_zzm5ki.jpg': 1.0}



In [47]:
cols =  ['xmin','ymin','xmax','ymax']
df[cols] = df[cols].astype(int)

In [49]:

augmentation_pipeline = A.Compose([

    # Color Adjustments
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.5),

    A.GaussianBlur(blur_limit=(3,9), p=0.3),
    A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=1,
                   shadow_dimension=5, shadow_intensity_range=(0.1,0.5),p=0.1),

    # Noise
    A.OneOf([
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
        A.ElasticTransform(alpha=10, sigma=10, p=0.3),
    ],p=1.),


    A.OneOf([
          A.Perspective(p=0.5, scale=(0.01, 0.1), keep_size=1, pad_mode=0, pad_val=(0, 0, 0), mask_pad_val=0, fit_output=0, interpolation=0),
          A.Affine(scale=(0.9, 1.1), translate_percent=(0.1, 0.2), rotate=(-15, 15), shear=(-5, 5), p=0.5),
        ],p=0.25),

    A.RandomResizedCrop(height=2048, width=2048, scale=(0.8, 1.0), p=0.1),

    A.LongestMaxSize(max_size=1024, p=1.),

], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

output_dir = "offline_aug_top80"
os.makedirs(output_dir, exist_ok=True)

augmented_labels = []
fix_seed(RANDOM_STATE)

for Image_ID,score in tqdm(candidate_image_ids.items()):
    d = df[df.Image_ID==Image_ID]
    image = cv2.imread(d.path.values[0])
    bboxes  = d[cols].values.tolist()
    labels = d['class'].values.tolist()
    n_aug = 3
    if score>=0.9:
      n_aug = 5
    elif score>=1.:
      n_aug = 10

    for i in range(n_aug):
        augmented = augmentation_pipeline(image=image, bboxes=bboxes, labels=labels)
        augmented_image = augmented['image']
        augmented_bboxes = augmented['bboxes']
        augmented_labels_list = augmented['labels']

        # Save augmented image
        aug_image_name = f"{Image_ID.split('.')[0]}_{i}.jpg"
        aug_image_path = os.path.join(output_dir, aug_image_name)
        cv2.imwrite(aug_image_path, augmented_image)

        # Store augmented labels
        for bbox, label in zip(augmented_bboxes, augmented_labels_list):
            augmented_labels.append({
                'image': aug_image_name,
                'xmin': bbox[0],
                'ymin': bbox[1],
                'xmax': bbox[2],
                'ymax': bbox[3],
                'label': label
            })



  1%|▏         | 18/1229 [01:36<1:07:36,  3.35s/it]/usr/local/lib/python3.10/dist-packages/albumentations/core/bbox_utils.py:478: RuntimeWarning: invalid value encountered in divide
  & (clipped_box_areas / denormalized_box_areas >= min_visibility - epsilon)
  3%|▎         | 32/1229 [02:52<1:17:58,  3.91s/it]/usr/local/lib/python3.10/dist-packages/albumentations/core/bbox_utils.py:478: RuntimeWarning: invalid value encountered in divide
  & (clipped_box_areas / denormalized_box_areas >= min_visibility - epsilon)
  3%|▎         | 37/1229 [03:08<1:19:00,  3.98s/it]/usr/local/lib/python3.10/dist-packages/albumentations/core/bbox_utils.py:478: RuntimeWarning: invalid value encountered in divide
  & (clipped_box_areas / denormalized_box_areas >= min_visibility - epsilon)
  3%|▎         | 38/1229 [03:10<1:06:17,  3.34s/it]/usr/local/lib/python3.10/dist-packages/albumentations/core/bbox_utils.py:478: RuntimeWarning: invalid value encountered in divide
  & (clipped_box_areas / denormalized_box

In [51]:
augmented_labels[0]

{'image': 'id_11ee1c_0.jpg',
 'xmin': 248.2196044921875,
 'ymin': 501.9608154296875,
 'xmax': 351.62353515625,
 'ymax': 548.1411743164062,
 'label': 'Pepper_Fusarium'}

In [64]:
df_augmented = pd.DataFrame(augmented_labels)
df_augmented.columns = ['Image_ID','xmin','ymin','xmax','ymax','class']
df_augmented['base_Image_ID'] = df_augmented.Image_ID.str.rsplit('_',n=1,expand=True)[0]+'.jpg'
df_augmented['path'] = f'{output_dir}/'+df_augmented.Image_ID
df_augmented.to_csv(f"{output_dir}/augmented_labels.csv", index=False)
df_augmented

,Image_ID,xmin,ymin,xmax,ymax,class,base_Image_ID,path
0,id_11ee1c_0.jpg,248.219604,501.960815,351.623535,548.141174,Pepper_Fusarium,id_11ee1c.jpg,offline_aug_top80/id_11ee1c_0.jpg
1,id_11ee1c_0.jpg,298.415680,387.764709,386.509796,445.490204,Pepper_Fusarium,id_11ee1c.jpg,offline_aug_top80/id_11ee1c_0.jpg
2,id_11ee1c_1.jpg,248.190689,501.962067,351.511963,548.090332,Pepper_Fusarium,id_11ee1c.jpg,offline_aug_top80/id_11ee1c_1.jpg
3,id_11ee1c_1.jpg,298.398865,387.662994,386.551819,445.259827,Pepper_Fusarium,id_11ee1c.jpg,offline_aug_top80/id_11ee1c_1.jpg
4,id_11ee1c_2.jpg,248.057632,501.946045,351.575104,548.161987,Pepper_Fusarium,id_11ee1c.jpg,offline_aug_top80/id_11ee1c_2.jpg
...,...,...,...,...,...,...,...,...
25739,id_zzm5ki_2.jpg,663.596375,500.649170,676.565857,519.440430,Pepper_Leaf_Blight,id_zzm5ki.jpg,offline_aug_top80/id_zzm5ki_2.jpg
25740,id_zzm5ki_3.jpg,651.656921,257.755005,673.988647,291.204742,Pepper_Leaf_Blight,id_zzm5ki.jpg,offline_aug_top80/id_zzm5ki_3.jpg
25741,id_zzm5ki_3.jpg,663.618774,500.663330,676.609070,519.450623,Pepper_Leaf_Blight,id_zzm5ki.jpg,offline_aug_top80/id_zzm5ki_3.jpg
25742,id_zzm5ki_4.jpg,651.736145,257.739410,674.028076,291.230377,Pepper_Leaf_Blight,id_zzm5ki.jpg,offline_aug_top80/id_zzm5ki_4.jpg


In [65]:
df_augmented.Image_ID.nunique()

5388

# Create YOLO format dataset

In [58]:
DATASET_DIRECTORY = 'datasets'
os.makedirs(DATASET_DIRECTORY,exist_ok=True)

In [59]:
df_val = df[df.fold==FOLD]
df_val

,Image_ID,confidence,class,ymin,xmin,ymax,xmax,path,category_id,crop,n_crop,fold
4,id_11ee1c.jpg,1.0,Pepper_Fusarium,2000,989,2184,1401,data/images/id_11ee1c.jpg,8,Pepper,1,0
5,id_11ee1c.jpg,1.0,Pepper_Fusarium,1545,1189,1775,1540,data/images/id_11ee1c.jpg,8,Pepper,1,0
6,id_11gglx.jpg,1.0,Corn_Cercospora_Leaf_Spot,774,2735,850,2834,data/images/id_11gglx.jpg,0,Corn,1,0
7,id_11gglx.jpg,1.0,Corn_Cercospora_Leaf_Spot,684,2204,735,2277,data/images/id_11gglx.jpg,0,Corn,1,0
8,id_11gglx.jpg,1.0,Corn_Cercospora_Leaf_Spot,1033,798,1092,880,data/images/id_11gglx.jpg,0,Corn,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
40999,id_pmnnbm.jpg,1.0,Pepper_Leaf_Mosaic,2023,2206,2183,2405,data/images/id_pmnnbm.jpg,13,Pepper,1,0
41010,id_yi3ay9.jpg,1.0,Pepper_Bacterial_Spot,2480,966,2521,1015,data/images/id_yi3ay9.jpg,5,Pepper,1,0
41016,id_kol3vy.jpg,1.0,Pepper_Septoria,78,2544,281,2760,data/images/id_kol3vy.jpg,14,Pepper,2,0
41017,id_i4msq6.jpg,1.0,Pepper_Septoria,747,1112,1081,1360,data/images/id_i4msq6.jpg,14,Pepper,2,0


In [70]:
df_aug = df_augmented.copy()
df_aug['fold'] = -1

# Filter validation set images to prevent leakage
# I'm leaving this here for consistency with the submitted model
# You can comment it out when training on the complete dataset
fold_image_ids = df_val.Image_ID.unique()
df_aug = df_aug[~df_aug.base_Image_ID.isin(fold_image_ids)].copy()
df_aug.shape

(20535, 9)

In [71]:
df_all = pd.concat([df,df_aug])
df_all['category_id'] = df_all['class'].map(CLASSES)
df_all

,Image_ID,confidence,class,ymin,xmin,ymax,xmax,path,category_id,crop,n_crop,fold,base_Image_ID
0,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,194.000000,328.000000,208.000000,341.000000,data/images/id_11543h.jpg,5,Pepper,1.0,3,NaN
1,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,149.000000,256.000000,162.000000,266.000000,data/images/id_11543h.jpg,5,Pepper,1.0,3,NaN
2,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,234.000000,327.000000,252.000000,338.000000,data/images/id_11543h.jpg,5,Pepper,1.0,3,NaN
3,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,221.000000,340.000000,238.000000,354.000000,data/images/id_11543h.jpg,5,Pepper,1.0,3,NaN
4,id_11ee1c.jpg,1.0,Pepper_Fusarium,2000.000000,989.000000,2184.000000,1401.000000,data/images/id_11ee1c.jpg,8,Pepper,1.0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25739,id_zzm5ki_2.jpg,NaN,Pepper_Leaf_Blight,500.649170,663.596375,519.440430,676.565857,offline_aug_top80/id_zzm5ki_2.jpg,11,NaN,NaN,-1,id_zzm5ki.jpg
25740,id_zzm5ki_3.jpg,NaN,Pepper_Leaf_Blight,257.755005,651.656921,291.204742,673.988647,offline_aug_top80/id_zzm5ki_3.jpg,11,NaN,NaN,-1,id_zzm5ki.jpg
25741,id_zzm5ki_3.jpg,NaN,Pepper_Leaf_Blight,500.663330,663.618774,519.450623,676.609070,offline_aug_top80/id_zzm5ki_3.jpg,11,NaN,NaN,-1,id_zzm5ki.jpg
25742,id_zzm5ki_4.jpg,NaN,Pepper_Leaf_Blight,257.739410,651.736145,291.230377,674.028076,offline_aug_top80/id_zzm5ki_4.jpg,11,NaN,NaN,-1,id_zzm5ki.jpg


In [72]:
df_all['split'] = 'train'
df_all.loc[df_all.fold==FOLD,'split'] = 'val'
df_all.split.value_counts()

,count
split,
train,50929
val,7637


In [ ]:
df['split'] = 'train'
df.loc[df.fold==FOLD,'split'] = 'val'
df.split.value_counts()

,count
split,
train,30394
val,7637


In [73]:
ID2CLASS = {v:k for k,v in CLASSES.items()}

conf = {
    'path': './',
    'train': 'train/images',
    'val': 'valid/images',
    'nc': len(CLASSES),
    'names': ID2CLASS
}
with open(f'{DATASET_DIRECTORY}/config.yaml','w') as f:
    yaml.dump(conf,f,default_flow_style=False)

display(conf)

{'path': './',
 'train': 'train/images',
 'val': 'valid/images',
 'nc': 23,
 'names': {0: 'Corn_Cercospora_Leaf_Spot',
  1: 'Corn_Common_Rust',
  2: 'Corn_Healthy',
  3: 'Corn_Northern_Leaf_Blight',
  4: 'Corn_Streak',
  5: 'Pepper_Bacterial_Spot',
  6: 'Pepper_Cercospora',
  7: 'Pepper_Early_Blight',
  8: 'Pepper_Fusarium',
  9: 'Pepper_Healthy',
  10: 'Pepper_Late_Blight',
  11: 'Pepper_Leaf_Blight',
  12: 'Pepper_Leaf_Curl',
  13: 'Pepper_Leaf_Mosaic',
  14: 'Pepper_Septoria',
  15: 'Tomato_Bacterial_Spot',
  16: 'Tomato_Early_Blight',
  17: 'Tomato_Fusarium',
  18: 'Tomato_Healthy',
  19: 'Tomato_Late_Blight',
  20: 'Tomato_Leaf_Curl',
  21: 'Tomato_Mosaic',
  22: 'Tomato_Septoria'}}

In [74]:
def normalize_labels(x,w = None, h=None):
    x = x.astype(np.float32)
    y = np.zeros_like(x).astype(np.float32)
    y[0] = (x[0] + x[2]) / 2 / w  # x center
    y[1] = (x[1] + x[3]) / 2 / h  # y center
    y[2] = (x[2] - x[0]) / w  # box width
    y[3] = (x[3] - x[1]) / h  # box height
    return y


In [76]:
def prep_ds(meta, split):
    out_dir = f'{DATASET_DIRECTORY}/{split}'
    shutil.rmtree(out_dir,ignore_errors=True)
    img_dir = f'{out_dir}/images'
    label_dir = f'{out_dir}/labels'
    os.makedirs(img_dir)
    os.makedirs(label_dir)

    # training on complete dataset to maintan consistency with the submitted model
    # comment for realistic validation
    if split=='train':
        df = meta.copy()
    else:
        df = meta[meta.split==split].copy()

    df.to_csv(f'{out_dir}/submission_format.csv',index=False)
    print(f'writing {split} data to {out_dir} images: {df.Image_ID.nunique()} annotations: {len(df)}')
    for image_id,d in tqdm(df.groupby('Image_ID')):
      path = d.path.values[0]
      d = d.dropna(subset=['xmin','ymin','xmax','ymax','category_id'])
      #skip bg only images
      if len(d)==0:
        print(f'skipping {image_id}')

      shutil.copy2(path, f"{img_dir}/{path.split('/')[-1]}")
      im = cv2.imread(path)
      height,width,_ = im.shape
      image_id = image_id.split('.')[0]
      for _, row in d.iterrows():
          if row.category_id==-1:
            continue
          x0, y0, x1, y1 = row.xmin,row.ymin,row.xmax,row.ymax
          w = x1 - x0
          h = y1 - y0
          x = np.array([x0,y0,x1,y1]) #['xmin', 'ymin', 'xmax', 'ymax']
          if w<1 or h<1:
            print(row)
            continue

          xnorm = normalize_labels(x,w=width,h=height)
          assert (xnorm>0).all()
          xnorm = xnorm.astype(str)
          # print(x,xnorm,height,width)
          with open(f"{label_dir}/{image_id}.txt",'a') as f:
              f.write(f"{int(row.category_id)}\t"+'\t'.join(xnorm)+'\n')

def gen_data(fold,meta):
    meta['split'] = 'train'
    meta.loc[meta.fold==fold,'split'] = 'valid'
    display(meta.split.value_counts())
    for split in ['train','valid']:
        prep_ds(meta,split)


gen_data(FOLD,df_all)

,count
split,
train,50929
valid,7637


writing train data to datasets/train images: 9198 annotations: 58566


 15%|█▍        | 1341/9198 [01:26<09:14, 14.17it/s]

Image_ID                           id_663pgs_0.jpg
confidence                                     NaN
class                          Tomato_Early_Blight
ymin                                    540.189636
xmin                                           0.0
ymax                                    576.371094
xmax                                      0.583861
path             offline_aug_top80/id_663pgs_0.jpg
category_id                                     16
crop                                           NaN
n_crop                                         NaN
fold                                            -1
base_Image_ID                        id_663pgs.jpg
split                                        train
Name: 3593, dtype: object


 15%|█▍        | 1361/9198 [01:28<07:44, 16.88it/s]

...

 97%|█████████▋| 8937/9198 [08:54<00:26,  9.83it/s]

Image_ID                     id_yux2jl.jpg
confidence                             1.0
class            Corn_Cercospora_Leaf_Spot
ymin                                  69.0
xmin                                1069.0
ymax                                  69.0
xmax                                1069.0
path             data/images/id_yux2jl.jpg
category_id                              0
crop                                  Corn
n_crop                                 1.0
fold                                     4
base_Image_ID                          NaN
split                                train
Name: 31765, dtype: object


100%|██████████| 9198/9198 [09:10<00:00, 16.71it/s]


writing valid data to datasets/valid images: 979 annotations: 7637


100%|██████████| 979/979 [01:25<00:00, 11.49it/s]


In [78]:
# !zip -rqq {DATASET_DIRECTORY}.zip {DATASET_DIRECTORY}